# check oof df

In [1]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity 
import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop import get_valid_values_dict, concat_valid_input_info, get_oof_df, get_key_df
from dss_model import get_model
from dss_dataloader import get_loader


In [2]:
output_dir = "/kaggle/working"
# exp_name = "exp022_dense_skffolds_epoch10"
# exp_name = "exp023_alldata_skf_dense_epoch30"
# exp_name = "exp025_alldata_skf_lstmenchead_epoch30"
# exp_name = "exp028_lstm_enchead_enckernelto48_epoch30"
# exp_name = "exp028_lstm_enchead_enckernelto96_epoch30"
# exp_name = "exp028_lstm_enchead_enckernelto192_epoch30"
# exp_name = "exp030_lstm_enchead_enckernelto384_removekeys_trnvld_epoch30"
exp_name = "exp040_downsample_epoch15_fold0"
# folds = [0, 1, 2, 3, 4]
folds = [0]
oof_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    # df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"raw_oof_df_fold{i}.parquet"))
    df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"fold{i}_best_oof_df.parquet"))
    oof_df = pd.concat([oof_df, df], axis=0)


print(len(oof_df["series_id"].unique()))   


fold 0
56


In [3]:
oof_df = oof_df.sort_values(["series_id", "step"])
display(oof_df)

oof_df = oof_df[["series_id", "series_date_key","step", "second", "event_pred", "class_pred", "anglez"]]


,series_id,step,anglez,enmo,second,date,anglez_mean_36,anglez_std_36,anglez_mean_60,anglez_std_60,event,event_onset,event_wakeup,series_date_key,series_date_key_str,fold,event_pred,class_pred,class_target
0,03d92c9f6f8a,0.0,38.892899,0.0803,0,2018-05-31,0.000000,0.000000,0.000000,0.000000,0.0,0,0,24,03d92c9f6f8a_2018-05-31,0,0.028784,0.036103,0.0
1,03d92c9f6f8a,12.0,-60.316101,0.0000,0,2018-05-31,0.000000,0.000000,0.000000,0.000000,0.0,0,0,24,03d92c9f6f8a_2018-05-31,0,0.000000,0.008165,0.0
2,03d92c9f6f8a,24.0,-88.147499,0.0114,0,2018-05-31,-62.547677,39.415679,0.000000,0.000000,0.0,0,0,24,03d92c9f6f8a_2018-05-31,0,0.000000,0.003050,0.0
3,03d92c9f6f8a,36.0,-88.216599,0.0000,0,2018-05-31,-79.737543,26.557696,-72.815246,32.900445,0.0,0,0,24,03d92c9f6f8a_2018-05-31,0,0.000000,0.003182,0.0
4,03d92c9f6f8a,48.0,-88.216599,0.0000,0,2018-05-31,-88.216599,0.000000,-83.129165,20.879465,0.0,0,0,24,03d92c9f6f8a_2018-05-31,0,0.000000,0.003324,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186945,fcca183903b7,620580.0,-89.679703,0.0127,0,2019-05-01,-88.890004,0.287931,-88.868442,0.243288,-1.0,0,0,7709,fcca183903b7_2019-05-01,0,0.000000,0.002627,-1.0
2186946,fcca183903b7,620592.0,-88.987297,0.0000,0,2019-05-01,-88.990804,0.247539,-88.928921,0.226916,-1.0,0,0,7709,fcca183903b7_2019-05-01,0,0.000000,0.002617,-1.0
2186947,fcca183903b7,620604.0,-88.987297,0.0000,0,2019-05-01,-88.969614,0.053042,-88.989401,0.190664,-1.0,0,0,7709,fcca183903b7_2019-05-01,0,0.000010,0.002614,-1.0
2186948,fcca183903b7,620616.0,-88.987297,0.0000,0,2019-05-01,-88.987297,0.000000,0.000000,0.000000,-1.0,0,0,7709,fcca183903b7_2019-05-01,0,0.000000,0.002612,-1.0


# nan detect

In [4]:
def detect_similarity_data(anglez, before_date_anglez, cut_window_num=4, only_cos_sim_thr=0.995):
    window_size = int(anglez.shape[1]/cut_window_num)
    count = 0
    for window_idx in range(cut_window_num):
        anglez_window = anglez[:,window_idx*window_size:(window_idx+1)*window_size]
        before_date_anglez_window = before_date_anglez[:, window_idx*window_size:(window_idx+1)*window_size]
        anglez_simirality = cosine_similarity(anglez_window, before_date_anglez_window)
        # only cos sim以上のsimilarityになるか、std_cos_sim以上のsimilarityかつstd_thr以下
        if anglez_simirality > only_cos_sim_thr:
            count += 1
    # 検出モデルに入力すべきものを1、スルーして無検出とすべきものを0にする
    detect_target = 1 - int(count > 0)
    return detect_target


In [5]:
def get_remove_target_keys(series_df, series_keys):
    remove_target_keys = []
    before_date_anglez = np.zeros((1, 1))
    loader = tqdm(enumerate(series_keys), total=len(series_keys))
    for idx, key in loader:
        # target_series = series_df.filter(pl.col("key")==key)
        target_series = series_df[series_df["series_date_key"]==key]
        # anglez = np.array(target_series.get_column("anglez")).reshape(1,-1)
        anglez = np.array(target_series["anglez"]).reshape(1,-1)
        if anglez.shape[1]==before_date_anglez.shape[1]:
            detect_pred = detect_similarity_data(anglez, before_date_anglez)
        else:
            detect_pred = 1
        if detect_pred==0:
            remove_target_keys.append(key)
        before_date_anglez = anglez
    return remove_target_keys


In [6]:
oof_df


,series_id,series_date_key,step,second,event_pred,class_pred,anglez
0,03d92c9f6f8a,24,0.0,0,0.028784,0.036103,38.892899
1,03d92c9f6f8a,24,12.0,0,0.000000,0.008165,-60.316101
2,03d92c9f6f8a,24,24.0,0,0.000000,0.003050,-88.147499
3,03d92c9f6f8a,24,36.0,0,0.000000,0.003182,-88.216599
4,03d92c9f6f8a,24,48.0,0,0.000000,0.003324,-88.216599
...,...,...,...,...,...,...,...
2186945,fcca183903b7,7709,620580.0,0,0.000000,0.002627,-89.679703
2186946,fcca183903b7,7709,620592.0,0,0.000000,0.002617,-88.987297
2186947,fcca183903b7,7709,620604.0,0,0.000010,0.002614,-88.987297
2186948,fcca183903b7,7709,620616.0,0,0.000000,0.002612,-88.987297


In [7]:
# series_keys = oof_df["series_date_key"].unique()
# # series_df = pl.DataFrame(oof_df)

# remove_target_keys = get_remove_target_keys(oof_df[["series_date_key", "anglez"]], series_keys)


  0%|          | 0/1587 [00:00<?, ?it/s]

In [8]:
# remove_target_keys


In [9]:
# pd.DataFrame({"remove_target_keys":remove_target_keys}).to_csv("/kaggle/working/remove_target_keys.csv", index=False)


In [10]:
remove_target_keys = pd.read_csv("/kaggle/working/remove_target_keys.csv")["remove_target_keys"].values.tolist()


# predict

In [26]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn

def detect_event_from_classpred(df, N=300, maxpool_kernel_size=41, remove_target_keys=None):
    df = df.copy()
    df = df[df["second"] == 0].reset_index(drop=True)

    # series_idでgroupbyして、class_predに対して対象の列のデータから
    # 前のN個の列までのデータの平均をとる
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x.rolling(N, min_periods=1).mean()
    )
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1]
    )
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - 1) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(
        maxpool_kernel_size, stride=1, padding=maxpool_padding
    )
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    # df.loc[df["series_date_key"].isin(remove_target_keys), "event_pred"] = 0
    df["onset"] = np.clip(-df["event_pred"], 0, 1)
    df["wakeup"] = np.clip(df["event_pred"], 0, 1)
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)

    return df

def make_submission_df(df, threshold=0.1):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [30]:
def detect_event_from_downsample_classpred(
    df, N=9, maxpool_kernel_size=3, maxpool_stride=1
):
    df = df.copy()
    df = df[df["second"] == 0].reset_index(drop=True)
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x.rolling(N, min_periods=1).mean()
    )
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1]
    )
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]
    not_predicted_mask = (df["class_pred"] != -1).astype(int)
    df["event_pred"] = df["event_pred"] * not_predicted_mask

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - maxpool_stride) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(
        maxpool_kernel_size, stride=maxpool_stride, padding=maxpool_padding
    )
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)
    return df


In [18]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["step"].notnull()].copy()
if len(folds)<5:
    train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()

train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())


,series_id,night,event,step,timestamp
0,03d92c9f6f8a,1,onset,5928.0,2018-05-31T20:14:00-0400
1,03d92c9f6f8a,1,wakeup,13524.0,2018-06-01T06:47:00-0400
2,03d92c9f6f8a,2,onset,23220.0,2018-06-01T20:15:00-0400
3,03d92c9f6f8a,2,wakeup,30276.0,2018-06-02T06:03:00-0400
4,03d92c9f6f8a,3,onset,40668.0,2018-06-02T20:29:00-0400


In [31]:
%%time
max_kernel_size = 3
tmp_score = 0
for average_size_ in range(3, 301, 3):
    print("average_size", average_size_, "max_kernel_size", max_kernel_size)
    # df = detect_event_from_classpred(oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size, remove_target_keys=remove_target_keys)
    df = detect_event_from_downsample_classpred(
        oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size
    )
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    print(len(sub_df))
    score_ = score(train_event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    average_size = average_size_


average_size 3 max_kernel_size 3
1893
0.629589270870827
average_size 6 max_kernel_size 3
2313
0.6797334830846601
average_size 9 max_kernel_size 3
2332
0.6854169528059091
average_size 12 max_kernel_size 3
2312
0.6871238625052567
average_size 15 max_kernel_size 3
2300
0.686300324530834
CPU times: user 11.3 s, sys: 40.6 ms, total: 11.3 s
Wall time: 10.7 s


In [14]:
tmp_score = 0
max_kernel_size_ = max_kernel_size
for max_kernel_size in range(31, 47, 2):
    print("average_size", average_size, "max_kernel_size", max_kernel_size)
    df = detect_event_from_classpred(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size, remove_target_keys=remove_target_keys)
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    score_ = score(train_event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    max_kernel_size_ = max_kernel_size


average_size 120 max_kernel_size 31


0.13102484640677245
average_size 120 max_kernel_size 33
0.13088510105454437


In [15]:
average_size = 132
max_kernel_size_ = 33
df = detect_event_from_classpred(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size_, remove_target_keys=remove_target_keys)
sub_df = make_submission_df(df, threshold=0.01)
print(average_size, max_kernel_size_)

if len(sub_df) == 0:
    print("event not detected")
    score_ = 0
else:
    score_ = score(train_event_df, sub_df)
print(score_)


132 33
0.1309924545033888


In [16]:
df = df[["series_id", "step", "class_pred", "onset", "wakeup"]].copy()
df = df.rename(columns={"class_pred": "sleep"})


In [17]:
print(df.columns)
os.makedirs(os.path.join(output_dir, "_overall_oof", exp_name), exist_ok=True)
df.to_parquet(os.path.join(output_dir, "_overall_oof", exp_name, "oof_df.parquet"))


Index(['series_id', 'step', 'sleep', 'onset', 'wakeup'], dtype='object')


exp023_alldata_skf_dense_epoch30

132 41

0.7210358824960511

exp025_alldata_skf_lstmenchead_epoch30
132 33
0.7500099409982081

exp030_lstm_enchead_enckernelto384_removekeys_trnvld_epoch30 132 33
0.7572738934042569